# Inferring gene regulatory networks (GRNs) with SCENIC

### Intro to SCENIC 

SCENIC is a tool to infer Gene Regulatory Networks and their associated cell states from **single-cell RNA-seq** data. 

A typical SCENIC workflow includes the following steps:

0. Setting up your dataset

Building the **gene regulatory network (GRN)**:

1. Identify potential targets for each TF based on co-expression ("co-expression modules").

    - Tool: GENIE3 or GRNBoost. 


2. Select potential direct-binding targets ("regulons") based on DNA-motif analysis 
    
    - Tool: RcisTarget

Identify **cell states** and their **regulators**:

3. Analyzing the network activity in each individual cell

    - Tool: AUCell
    - Scoring regulons in the cells (calculate AUC)
    - Optional: Convert the network activity into ON/OFF (binary activity matrix)

4. Identify stable cell states based on their gene regulatory network activity (cell clustering) 
and exploring the results...

   ![SCENIC.png](SCENIC.png)

For more details on how and why SCENIC folows these steps, and some usage examples, you can check the original [paper](https://www.nature.com/articles/nmeth.4463).  

To run SCENIC on a new dataset, we would recommend you to check the [requirements](https://rawcdn.githack.com/aertslab/SCENIC/66656c71f99000a67d3f25e8b811e18338ff8270/inst/doc/SCENIC_Setup.html) (for example, at the moment it is only available for mouse, human and fly). 


### Implementation: 

There are currently implementations of SCENIC in [R](https://github.com/aertslab/SCENIC/) and in [Python](https://github.com/aertslab/pySCENIC). For running it in batch  on multiple datasets, or on big datasets, we normally recommend to use **[VSN](https://github.com/vib-singlecell-nf/vsn-pipelines)** (a *Nextflow workflow*), which highly automates and simplifies the analysis, as explained in the [SCENIC protocol paper](https://doi.org/10.1038/s41596-020-0336-2) and [examples](https://github.com/aertslab/SCENICprotocol/). 
The output from VSN can be explored from either R or Python, and the web browser (SCope).
 
In this workshop, we will run SCENIC using **VSN** (Notebook 1), and explore the output in **SCope and R** (Notebook 2):

- Notebook 1: Running SCENIC (VSN: includes Steps 1-4)

- Notebook 2: Exploring SCENIC's output (R)

For advanced users who might want more details, modify the algorithm, or run only parts of it, detailed tutorials explaining each step in deatail are avilable in R (e.g. 
[step by step](https://rawcdn.githack.com/aertslab/SCENIC/6aed5ef0b0386a87982ba4cc7aa13db0444263a6/inst/doc/SCENIC_Running.html ) and explanation of source code for [regulons](https://github.com/aertslab/SCENIC/blob/master/vignettes/detailedStep_2_createRegulons.Rmd)).


----

*Where to find help:*

When you are doing your own analysis, you will likely bump into problems. In that case: 

* **Read the error message**: most of the time they are self-explanatory
* Check the **help page of the function** you're using (e.g. `?get_regulons`)
* Google that error message: if it's not clear to you what the problem is, just Google it; scRNA-seq community is very active (e.g. on BioStars or on stackoverflow).
* Find and follow the package [**tutorials**](https://github.com/aertslab/SCENIC) (in R they are called "vignettes"). 


##  Notebook 1: Running SCENIC

### Running SCENIC through VSN

This part of the workshop is based on the
[VSN pipeline tutorial](https://vsn-pipelines-examples.readthedocs.io/en/latest/PBMC10k.html).

For more complex examples, i.e. explaing gene filtering, see the SCENIC [protocol example](https://htmlpreview.github.io/?https://github.com/aertslab/SCENICprotocol/blob/master/notebooks/PBMC10k_SCENIC-protocol-CLI.html)

#### Before starting...

1. Check system requirements: VSN requires `Nextflow` and a container system (`Singularity` or `Docker`). VSN is run directly on a terminal.

> Make sure to have matching versions. e.g. at the time of preparing this notebook: `VSN 0.24.0 -> Nextflow 20.04.1`
and `VSN 0.25.0 -> Nextflow 20.10.1+`

In [39]:
date

date
Tue Feb  9 13:19:06 CET 2021


: 1

In [2]:
module load Nextflow/20.10.0 # only needed in some systems
nextflow -version


      N E X T F L O W
      version 20.10.0 build 5431
      created 01-11-2020 15:28 UTC (16:28 CEST)
      cite doi:10.1038/nbt.3820
      http://nextflow.io



In [4]:
singularity version 

3.6.4-1.el7


Also make sure that LANG and LC_ALL environment variables have been set:

In [6]:
locale

LANG=en_US.utf8
LC_CTYPE="en_US.utf8"
LC_NUMERIC="en_US.utf8"
LC_TIME="en_US.utf8"
LC_COLLATE=C
LC_MONETARY="en_US.utf8"
LC_MESSAGES="en_US.utf8"
LC_PAPER="en_US.utf8"
LC_NAME="en_US.utf8"
LC_ADDRESS="en_US.utf8"
LC_TELEPHONE="en_US.utf8"
LC_MEASUREMENT="en_US.utf8"
LC_IDENTIFICATION="en_US.utf8"
LC_ALL=


In [8]:
# If some are not set, you can set them to the default language for instance:
export LANG="C"
export LC_ALL="C"

2. Set your working directory:

In [10]:
# ln -s /anywhere/Projects/Tutorials/ ~
cd ~/Tutorials/

In [12]:
mkdir -p SCENIC_pbmc
cd SCENIC_pbmc

#### 1. Prepare dataset

This tutorial analyzes a typical scRNA-seq dataset with a single run: **Blood cells (PBMC)** data available from the 10x Genomics support website. 

- The “Feature / cell matrix (filtered)” data can be downloaded from 10x Genomics website (It is the typical output from their pipeline, after running Cell Ranger)

- This dataset includes **10k cells**. It would take approximately **2 hours** to run SCENIC on it with a HPC system using 15 processes. Therefore, in this tutorial we will not *run* the analysis. Instead, we will prepare the inputs, and in the second part of the tutorial, we will directly explore the .loom file that was produced as output from a previous run.

In [14]:
wget http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz

--2021-02-09 13:02:03--  http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 2606:4700::6812:ad, 2606:4700::6812:1ad, 104.18.1.173, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|2606:4700::6812:ad|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz [following]
--2021-02-09 13:02:03--  https://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|2606:4700::6812:ad|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94334700 (90M) [application/x-tar]
Saving to: 'pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz.5'

100%[======================================>] 94,334,700  70.0MB/s   in 1.3s   

2021-02-09 13:0

When using 10x data as an input, the pipeline assumes the files are in the typical Cell Ranger directory structure (`<datasetName>/outs/`). Therefore, we will extract the files into a folder following that naming scheme:

In [16]:
# Create directory structure:
mkdir -p pbmc10k_data/outs/
# Extract the file:
tar xvf pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz -C pbmc10k_data/outs/

filtered_feature_bc_matrix/
filtered_feature_bc_matrix/matrix.mtx.gz
filtered_feature_bc_matrix/barcodes.tsv.gz
filtered_feature_bc_matrix/features.tsv.gz


In [18]:
# See the resulting files:
tree pbmc10k_data

pbmc10k_data
`-- outs
    `-- filtered_feature_bc_matrix
        |-- barcodes.tsv.gz
        |-- features.tsv.gz
        `-- matrix.mtx.gz

2 directories, 3 files


In the following step, when setting up the nextflow config file, the `tenx` input channel should point to the outs folder: `pwd + 'pbmc10k_data/outs'`

#### 2. Setup the `VSN-pipelines` project

First, pull the VSN repository in Nextflow. The `-r` flag can be used to specify the pipeline version to use:

In [20]:
nextflow pull vib-singlecell-nf/vsn-pipelines -r v0.25.0

Checking vib-singlecell-nf/vsn-pipelines ...
done - revision: 721c42f889 [v0.25.0]


Then, define the run **settings** (a.k.a. create the `config` file):

- `-profile`: Determines the parameters to include in the config file. Select the pipelines/settings that will be used in this specific analysis (i.e. this argument is to avoid creating huge file with *all* possibilities for *all* VSN-pipelines).

In this case, we have used:

- `tenx`: defines the input data type
- `single_sample_scenic`: creates the basic parameters for the `single_sample` and `scenic` workflows (see https://vsn-pipelines.readthedocs.io/en/latest/pipelines.html for the list of available pipelines and their options)
- `scenic_use_cistarget_motifs` and `scenic_use_cistarget_track`: includes parameters to specify the location of the cistarget database files (modify their location in the config file)
- `hg38`: specifies the genome
- `singularity` (or `docker`): specifies container system to use to run the processes

In [22]:
nextflow config vib-singlecell-nf/vsn-pipelines \
    -profile tenx,single_sample_scenic,scenic_use_cistarget_motifs,scenic_use_cistarget_tracks,hg38,singularity \
    > pbmc10k.vsn-pipelines.complete.config

This will create the config file: `pbmc10k.vsn-pipelines.complete.config`

Important variables to check/edit in the config file are:

- `singularity.runOption` (or `docker.runOptions`): the correct volume mounts should be specified (requires the user home folder, and the location of the data).
- `params.global.project_name` (optional): will control the naming of the output files.
- `params.sc.scope.tree.level_${X}` (optional): controls the labeling of the loom file when uploaded to the SCope viewer.
- `params.sc.scanpy.filter`
- `params.sc.scanpy.feature_selection`
- `params.sc.scanpy.clustering`

*Exercise:* 
Open the config file and edit the settings required to continue the tutorial, including: 
- the project name
- `cellranger_mex = 'pbmc10k_data/outs/'`

#### 3. Launch the run

**First pass: Check filtering settings**

While the overall goal is to run SCENIC, the VSN pipeline also includes preprocessing and filtering steps.
Those filtering settings should be checked to confirm they are the appropriate for the given dataset.

To save running time, it is possible and recommended to make a first pass running only those pre-processing steps  (determined by the argument `-entry single_sample`).
i.e.:

In [24]:
nextflow -C pbmc10k.vsn-pipelines.complete.config \
    run vib-singlecell-nf/vsn-pipelines \
    -entry single_sample \
    -r v0.25.0

N E X T F L O W  ~  version 20.10.0
Launching `vib-singlecell-nf/vsn-pipelines` [trusting_stallman] - revision: 721c42f889 [v0.25.0]
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE_SAMPLE... -
[-        ] process > single_sample:SINGLE

: 1

> The argument `-entry` determines the pipeline that will be run. Note that it will ony be possible to run the pipelines added to the *config file* in the previous step. In this case 'single_sample_scenic' is equivalent to running 'single_sample' + SCENIC. See [VSN's readthedocs](
https://vsn-pipelines.readthedocs.io/en/latest/pipelines.html) for the list and description of the different pipelines available.

The resulting **QC reports** will be located in `out/notebooks/intermediate/pbmc10k.SC_QC_filtering_report.htm` (as ipynb, and converted html file). 

If needed, the cell and gene filters can be updated by editing the config file. 

For example, the filters used by default are:

```
params {
    sc {
        scanpy {
            filter = {
                cellFilterMinNGenes = 200
                cellFilterMaxNGenes = 4000
                cellFilterMaxPercentMito = 0.15
                geneFilterMinNCells = 3
            }
        }
    }
}
```

*Exercise:* Open the QC reports and have a look at the stats provided.

**Second pass: Run SCENIC**

Once the cell and gene filters look fine, we can re-start the pipeline to run SCENIC (setting `-entry single_sample_scenic`). 

Using  the `-resum` option will continue running the pipeline, skipping already completed steps: In this case, applying the correct filtering parameters, and continue to the upcoming SCENIC steps.

#### Results & output

The main SCENIC outputs (including regulons and cell projections based on regulon activity) are packaged into a **loom file**. The loom file also includes the results of the parallel expression analysis. 

- The loom file can be found at `out/loom/pbmc10k.SCENIC_SCope_output.loom`, and is ready to be uploaded to a [SCope](http://scope.aertslab.org/) session. 
(The loom file from this analysis can be found on the [SCENIC protocol SCope session](https://scope.aertslab.org/#/Protocol_Cases/) (DSL2 > pbmc10k.SCENIC_SCope_output.loom).

Other relevant output files include:

- `out/scenic/pbmc10k_data/notebooks/SCENIC_report.html`: Notebook with an overview of the SCENIC workflow. The last heatmap ("AUC Heatmap - Top 5 regulons from each cell type"), provides an useful overview of the regulons in the cells.

- `out/scenic/`: The other folders contain partial results from SCENIC pipeline, which can be useful for advanced users. For example, the **motif enrichment analysis** and **co-expression modules** (output of GRNBoost/GENIE3).

- `out/data/pbmc10k.PBMC10k_DSL2.single_sample.output.h5a`: an anndata file generated by the **Scanpy** section of the pipeline, including the results of the expression analysis in addition to SCENIC(i.e. clustering based on highly variable genes).

- The `work` folder contains temporary files to allow resuming the pipeline if needed. It can be deleted once the pipeline is finished.

In the second Notebook, we will explore these outputs in SCope and R.

To see the list of files:

In [27]:
tree out

tree out
out
|-- data
|   |-- intermediate
|   |   |-- pbmc10k_data.10x_PBMC.single_sample.final_output.h5ad -> /ddn1/vol1/staging/leuven/stg_00002/lcb/saibar/Projects/Tutorials/SCENIC_pbmc_10k_run2/work/34/43391af2c86d9fddfb1a066811bd8b/pbmc10k_data.10x_PBMC.single_sample.final_output.h5ad
|   |   |-- pbmc10k_data.SCANPY.hvg_scaled_output.h5ad -> /ddn1/vol1/staging/leuven/stg_00002/lcb/saibar/Projects/Tutorials/SCENIC_pbmc/work/e9/30e3fd13ffa8c31a6fbed9d7497598/pbmc10k_data.SCANPY.hvg_scaled_output.h5ad
|   |   |-- pbmc10k_data.SCANPY.normalized_output.h5ad -> /ddn1/vol1/staging/leuven/stg_00002/lcb/saibar/Projects/Tutorials/SCENIC_pbmc/work/43/13ba9c99b8e9e3ca9ac11adf5f5d9e/pbmc10k_data.SCANPY.normalized_output.h5ad
|   |   |-- pbmc10k_data.SC__FILE_CONVERTER.h5ad -> /ddn1/vol1/staging/leuven/stg_00002/lcb/saibar/Projects/Tutorials/SCENIC_pbmc/work/de/71393df76480af5321b896bd2a4a34/pbmc10k_data.SC__FILE_CONVERTER.h5ad
|   |   |-- pbmc10k_data.SC__H5AD_MERGE.h5ad -> /ddn1/vol1/staging

: 1